# Time calculations in PyDANDIA

Originally, pyDANDIA made use of the SLAlib library to calculate Heliocentric Julian Date as it was available as a Python-wrapped C-library and was a well-tested standard.  However, it is a difficult package to reliably install and build, and there are Astropy functions available which should be easier to use.  The purpose of this notebook is to trial an Astropy-based replacement for the SLAlib routines in PyDANDIA's time_utils.py

Support for HJD was added in Astropy as discussed in this thread:
https://github.com/astropy/astropy/pull/4314

In [96]:
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time, TimeDelta
from astropy import units as u
import math
from datetime import datetime, timedelta
import pyslalib.slalib as S

# Astropy implementation of HJD calculation

Based on the [Astropy documentation](https://docs.astropy.org/en/stable/time/), the following function calculates HJD:

In [97]:
# Astropy requires that an observatory site is specified
observatory_site = EarthLocation.of_site('lapalma')
#observatory_site = EarthLocation.from_geocentric(0,0,0, unit=u.m)

t = Time("2024-03-05T14:00", format='isot', scale='utc', location=observatory_site)
print('Measured timestamp: ' + str(t))

star = SkyCoord('17:30:00', '-27:00:00', unit=(u.hourangle,u.degree), frame='icrs')
print('Star coordinates in degrees: ',star.ra.deg, star.dec.deg)
print('Star coordinates in rads: ', star.ra.rad, star.dec.rad)

# This uses EPV rather than ECOR by default, meaning it is more precise than the low-precision SLAlib
# Applying the JPL ephemeris makes it more precise still
ltt_helio = t.light_travel_time(star, 'heliocentric', ephemeris=None) 
print('Light travel time (Earth-Sun) at this timestamp: ' + str(ltt_helio*24.0*60*60) + 'sec')

hjd1 = t.utc + ltt_helio
print('Heliocentric-corrected timestamp: ' + str(hjd1) + ', ' + str(hjd1.jd))
hjd1

Measured timestamp: 2024-03-05T14:00:00.000
Star coordinates in degrees:  262.49999999999994 -27.0
Star coordinates in rads:  4.581489286485114 -0.47123889803846897
Light travel time (Earth-Sun) at this timestamp: -70.05597380191763sec
Heliocentric-corrected timestamp: 2024-03-05T13:58:49.944, 2460375.0825225003


<Time object: scale='utc' format='isot' value=2024-03-05T13:58:49.944>

## SLAlib comparison

Now calculate the HJD using the [SLAlib-based routine from pyDANDIA](https://github.com/pyDANDIA/pyDANDIA/blob/feature/quick_phot/pyDANDIA/time_utils.py).

In [98]:
def sexig2dec(coord):
    """Function to convert a sexigesimal coordinate string into a decimal float,
    returning a value in the same units as the string passed to it."""

    # Ensure that the string is separated by ':':
    coord = coord.lstrip().rstrip().replace(' ',':')

    # Strip the sign, if any, from the first character, making a note if its negative:
    if coord[0:1] == '-':
      	sign = -1.0
      	coord = coord[1:]
    else:
      	sign = 1.0

    # Split the CoordStr on the ':':
    coord_list = coord.split(':')

    # Assuming this presents us with a 3-item list, the last two items of which will
    # be handled as minutes and seconds:
    try:
        decimal = float(coord_list[0]) + (float(coord_list[1])/60.0) + \
                            (float(coord_list[2])/3600.0)
        decimal = sign*decimal

    except:
        decimal = 0

    # Return with the decimal float:
    return decimal


In [99]:
def datetime2mjd_utc(d):

# Compute MJD for UTC
    (mjd, status) = S.sla_cldj(d.year, d.month, d.day)
    if status != 0:
        return None
    (fday, status ) = S.sla_dtf2d(d.hour, d.minute, d.second+(d.microsecond/1e6))
    if status != 0:
        return None
    mjd_utc = mjd + fday

    return mjd_utc

In [100]:
def mjd_utc2mjd_tt(mjd_utc, dbg=False):
    '''Converts a MJD in UTC (MJD_UTC) to a MJD in TT (Terrestial Time) which is
    needed for any position/ephemeris-based calculations.
    UTC->TT consists of: UTC->TAI = 10s offset + 24 leapseconds (last one 2009 Jan 1.)
    	    	    	 TAI->TT  = 32.184s fixed offset'''
# UTC->TT offset
    tt_utc = S.sla_dtt(mjd_utc)
    if dbg:
        print('TT-UTC(s)='+str(tt_utc))

# Correct MJD to MJD(TT)
    mjd_tt = mjd_utc + (tt_utc/86400.0)
    if dbg:
        print('MJD(TT)  =  '+str(mjd_tt))

    return mjd_tt


In [101]:
def calc_hjd(dateobs,RA,Dec,exptime,debug=False):
    """Function to calculate the Heliocentric Julian Date from the parameters
    in a typical image header:

    :params string dateobs: DATE-OBS, Exposure start time in UTC,
                            %Y-%m-%dT%H:%M:%S format
    :params float exptime:  Exposure time in seconds
    :params string RA:      RA in sexigesimal hours format
    :params string Dec:     Dec in sexigesimal degrees format

    Returns:

    :params float HJD:      HJD
    """

    # Convert RA, Dec to radians: then convert from J2000 to mean position, epoch of date -> FK5, J2000.0
    dRA = sexig2dec(RA)
    dRA = dRA * 15.0 * math.pi / 180.0
    dDec = sexig2dec(Dec)
    dDec = dDec * math.pi / 180.0
    #dRA = 4.588083085411322  # Radians, in mean position at dateobs
    #dDec = -0.47153769281650487
    if debug:
        print('RA '+RA+' -> decimal radians '+str(dRA))
        print('Dec '+Dec+' -> decimal radians '+str(dDec))

    # Convert the timestamp into a DateTime object:
    if 'T' in dateobs:
        try:
            dt = datetime.strptime(dateobs,"%Y-%m-%dT%H:%M:%S.%f")
        except ValueError:
            dt = datetime.strptime(dateobs,"%Y-%m-%dT%H:%M:%S")
    else:
        try:
            dt = datetime.strptime(dateobs,"%Y-%m-%d %H:%M:%S.%f")
        except ValueError:
            dt = datetime.strptime(dateobs,"%Y-%m-%d %H:%M:%S")

    # Convert the exposure time into a TimeDelta object and add half of it
    # to the time to get the exposure mid-point:
    expt = timedelta(seconds=exptime)

    dt = dt + expt/2.0

    if debug:
        print('DATE-OBS = '+str(dateobs))
        print('Exposure time = '+str(expt))
        print('Mid-point of exposure = '+dt.strftime("%Y-%m-%dT%H:%M:%S.%f"))

        at = Time(dateobs,format='isot', scale='utc')
        aexpt = TimeDelta(exptime,format='sec')

        adt = at + aexpt/2.0
        print('Astropy: mid-point of exposure = '+adt.value)

    # Calculate the MJD (UTC) timestamp:
    mjd_utc = datetime2mjd_utc(dt)
    if debug:
        print('MJD_UTC = '+str(mjd_utc))
        print('Astropy MJD_UTC = '+str(adt.mjd))

    # Correct the MJD to TT:
    mjd_tt = mjd_utc2mjd_tt(mjd_utc)
    if debug:
        print('MJD_TT = '+str(mjd_tt))

        att = adt.tt
        print('Astropy MJD_TT = '+str(att.mjd))

    # Correct coordinates to the mean position at a set date
    obs_epoch = S.sla_epj(mjd_tt)
    dRA_fk5, dDec_fk5 = S.sla_preces('FK5', 2000.0, obs_epoch, dRA, dDec)
    print('Corrected mean position: ', dRA_fk5, dDec_fk5)
    
    # Calculating MJD of 1st January that year: sla_clyd XXXX convert to TT?
    (mjd_jan1,iexec) = S.sla_cldj(dt.year,1,1)
    if debug:
        print('MJD of Jan 1, '+str(dt.year)+' = '+str(mjd_jan1))

        at_jan1 = Time(str(dt.year)+'-01-01T00:00:00.0',format='isot', scale='utc')
        print('Astropy MJD of Jan 1, '+str(dt.year)+' = '+str(at_jan1.mjd))

    # Calculating the MJD difference between the DateObs and Jan 1 of the same year:
    tdiff = mjd_tt - mjd_jan1
    if debug:
        print('Time difference from Jan 1 - dateobs, '+\
                str(dt.year)+' = '+str(tdiff))

        atdiff = att.mjd - at_jan1.mjd
        print('Astropy time difference = '+str(atdiff))

    # Calculating the RV and time corrections to the Sun: XX Year could change, should use TDB
    print(dRA,dDec,dt.year,int(tdiff),(tdiff-int(tdiff)))
    (rv,tcorr) = S.sla_ecor(dRA_fk5,dDec_fk5,dt.year,int(tdiff),(tdiff-int(tdiff)))
    if debug:
        print('Time correction to the Sun = '+str(tcorr))

    # Calculating the HJD:
    hjd = mjd_tt + tcorr/86400.0 + 2400000.5
    if debug:
        print('HJD = '+str(hjd))

    iy, im, id, fd, stat = S.sla_djcl(mjd_tt)
    print(iy, im, id, fd, stat)
    ny, nd, stat = S.sla_clyd(iy, im, id)
    print(ny, nd)
    (rv2,tcorr2) = S.sla_ecor(dRA_fk5,dDec_fk5,ny,nd,fd)
    print('New time correction: ' , tcorr2)
    
    return hjd, tcorr2

In [102]:
dateobs = '2024-03-05T14:00:00'
ra = '17:30:00'
dec = '-27:00:00'
exptime = 0.0
hjd2, tcorr2 = calc_hjd(dateobs,ra,dec,exptime,debug=True)

RA 17:30:00 -> decimal radians 4.581489286485115
Dec -27:00:00 -> decimal radians -0.47123889803846897
DATE-OBS = 2024-03-05T14:00:00
Exposure time = 0:00:00
Mid-point of exposure = 2024-03-05T14:00:00.000000
Astropy: mid-point of exposure = 2024-03-05T14:00:00.000
MJD_UTC = 60374.583333333336
Astropy MJD_UTC = 60374.583333333336
MJD_TT = 60374.58413407407
Astropy MJD_TT = 60374.58413407407
Corrected mean position:  4.588083339638706 -0.471537830374264
MJD of Jan 1, 2024 = 60310.0
Astropy MJD of Jan 1, 2024 = 60310.0
Time difference from Jan 1 - dateobs, 2024 = 64.58413407407352
Astropy time difference = 64.58413407407352
4.581489286485115 -0.47123889803846897 2024 64 0.5841340740735177
Time correction to the Sun = -78.54212951660156
HJD = 2460375.0832250216
2024 3 5 0.5841340740735177 0
2024 65
New time correction:  -70.02302551269531


In [103]:
print('Difference HJD1 - HJD2 = ' + str((hjd2 - hjd1.jd)*24.0*60.0) + 'm')

Difference HJD1 - HJD2 = 1.0116306692361832m


In [104]:
t1 = .583333333336
t2 = .58413407407
print((t2 - t1)*86400)

69.18399941759468


In [105]:
ltt_helio.to(u.s).value - tcorr2

-0.032948289222318294

In [106]:
ogg = EarthLocation(lat='20d42m25.5sN', lon='156d15m27.4sW', height=3055.0 * u.m)
ogg

<EarthLocation (-5466072.06175456, -2404262.63207449, 2242169.36230874) m>

In [112]:
def fetch_observatory_location(tel_code):
    lco_facilities = {
        'ogg-clma-2m0a': EarthLocation(lat='20d42m25.5sN', lon='156d15m27.4sW', height=3055.0 * u.m),
        'ogg-clma-0m4b': EarthLocation(lat='20d42m25.1sN', lon='156d15m27.11sW', height=3037.0 * u.m),
        'ogg-clma-0m4c': EarthLocation(lat='20d42m25.1sN', lon='156d15m27.12sW', height=3037.0 * u.m),
        'coj-clma-2m0a': EarthLocation(lat='31d16m23.4sS', lon='149d4m13.0sE', height=1111.8 * u.m),
        'coj-doma-1m0a': EarthLocation(lat='31d16m22.56sS', lon='149d4m14.33sE', height=1168.0 * u.m),
        'coj-domb-1m0a': EarthLocation(lat='31d16m22.89sS', lon='149d4m14.75sE', height=1168.0 * u.m),
        'coj-clma-0m4a': EarthLocation(lat='31d16m22.38sS', lon='149d4m15.05sE', height=1191.0 * u.m),
        'coj-clma-0m4b': EarthLocation(lat='31d16m22.48sS', lon='149d4m14.91sE', height=1191.0 * u.m),
        'elp-doma-1m0a': EarthLocation(lat='30d40m47.53sN', lon='104d0m54.63sW', height=2010.0 * u.m),
        'elp-domb-1m0a': EarthLocation(lat='30d40m48.00sN', lon='104d0m55.74sW', height=2029.4 * u.m),
        'elp-aqwa-0m4a': EarthLocation(lat='30d40m48.15sN', lon='104d0m54.24sW', height=2027.0 * u.m),
        'lsc-doma-1m0a': EarthLocation(lat='30d10m2.58sS', lon='70d48m17.24sW', height=2201.0 * u.m),
        'lsc-domb-1m0a': EarthLocation(lat='30d10m2.39sS', lon='70d48m16.78sW', height=2201.0 * u.m),
        'lsc-domc-1m0a': EarthLocation(lat='30d10m2.81sS', lon='70d48m16.85sW', height=2201.0 * u.m),
        'lsc-aqwa-0m4a': EarthLocation(lat='30d10m3.79sS', lon='70d48m16.88sW', height=2202.5 * u.m),
        'lsc-aqwb-0m4a': EarthLocation(lat='30d10m3.56sS', lon='70d48m16.74sW', height=2202.5 * u.m),
        'cpt-doma-1m0a': EarthLocation(lat='32d22m50.0sS', lon='20d48m36.65sE', height=1807.0 * u.m),
        'cpt-domb-1m0a': EarthLocation(lat='32d22m50.0sS', lon='20d48m36.13sE', height=1807.0 * u.m),
        'cpt-domc-1m0a': EarthLocation(lat='32d22m50.38sS', lon='20d48m36.39sE', height=1807.0 * u.m),
        'cpt-aqwa-0m4a': EarthLocation(lat='32d22m50.25sS', lon='20d48m35.54sE', height=1804.0 *u.m),
        'tfn-doma-1m0a': EarthLocation(lat='28d18m1.56sN', lon='16d30m41.82sE', height=2406.0 * u.m),
        'tfn-domb-1m0a': EarthLocation(lat='28d18m1.8720sN', lon='16d30m41.4360sE', height=2406.0 * u.m),
        'tfn-aqwa-0m4a': EarthLocation(lat='28d18m1.11sN', lon='16d30m42.13sE', height=2390.0 * u.m),
        'tfn-aqwa-0m4a': EarthLocation(lat='28d18m1.11sN', lon='16d30m42.21sE', height=2390.0 * u.m),
    }
    
    # Check to see if it is an LCO facility code first, because we have more precise 
    # coordinates for each telescope than a generic site location
    if tel_code in lco_facilities.keys():
        return lco_facilities[tel_code]

    # If not found, use Astropy's built-in look-up table.  If this is fed an invalid 
    # site ID code, it will automatically raise an UnknownSiteException, which is
    # the desired behaviour
    else:
        return EarthLocation.of_site(tel_code)
        

In [113]:
obs1 = fetch_observatory_location('ogg-clma-2m0a')
obs1

obs2 = fetch_observatory_location('foo')
obs2

{'ogg-clma-2m0a': <EarthLocation (-5466072.06175456, -2404262.63207449, 2242169.36230874) m>, 'ogg-clma-0m4b': <EarthLocation (-5466057.25271517, -2404265.29017691, 2242151.48536641) m>, 'ogg-clma-0m4c': <EarthLocation (-5466057.36927723, -2404265.02517498, 2242151.48536641) m>, 'coj-clma-2m0a': <EarthLocation (-4681246.77022575, 2804967.64212724, -3292393.97389006) m>, 'coj-doma-1m0a': <EarthLocation (-4681317.58278984, 2804969.04994609, -3292401.03259024) m>, 'coj-domb-1m0a': <EarthLocation (-4681318.76769187, 2804956.80549828, -3292409.72093929) m>, 'coj-clma-0m4a': <EarthLocation (-4681346.70588824, 2804964.29233773, -3292408.23312442) m>, 'coj-clma-0m4b': <EarthLocation (-4681343.43034414, 2804966.64785303, -3292410.86596925) m>, 'elp-doma-1m0a': <EarthLocation (-1330025.63799311, -5328428.97615206, 3236445.76104197) m>, 'elp-domb-1m0a': <EarthLocation (-1330056.56415159, -5328430.83892287, 3236468.11170033) m>, 'elp-aqwa-0m4a': <EarthLocation (-1330016.74381361, -5328436.22104676

UnknownSiteException: "Site 'foo' not in database. Use EarthLocation.get_site_names to see available sites. If 'foo' exists in the online astropy-data repository, use the 'refresh_cache=True' option to download the latest version. Did you mean one of: 'oao'?'"

In [115]:
faccode = 'ogg-clma-2m0a-fs02'

'-'.join(faccode.split('-')[0:3])

'ogg-clma-2m0a'